#  PHASE 6: Cost-Sensitive Evaluation

## Big-Tech-Grade User Retention & Churn Prediction System

---

**Author**: Senior Data Scientist  
**Date**: February 2026  
**Objective**: Optimize decision threshold based on business costs, not just ML metrics

---

##  This is the MOST IMPORTANT Phase

A model with great ML metrics can still destroy business value if the threshold is wrong!

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import precision_recall_curve, roc_curve, confusion_matrix
import joblib
import os
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries imported")

In [ ]:
PROJECT_ROOT = '/Users/anuj/Desktop/Churn_Retension/churn-prediction-bigtech'
PROCESSED_DATA_PATH = os.path.join(PROJECT_ROOT, 'data', 'processed')
MODELS_PATH = os.path.join(PROJECT_ROOT, 'models')

test_df = pd.read_parquet(os.path.join(PROCESSED_DATA_PATH, 'test.parquet'))
target_col = 'is_churned'
feature_cols = [c for c in test_df.columns if c != target_col]
X_test, y_test = test_df[feature_cols], test_df[target_col]

try:
    model = joblib.load(os.path.join(MODELS_PATH, 'xgboost_final.pkl'))
    print("✅ Loaded XGBoost model")
except:
    model = joblib.load(os.path.join(MODELS_PATH, 'random_forest.pkl'))
    print("✅ Loaded Random Forest model")

y_proba = model.predict_proba(X_test)[:, 1]

---

## 1. Define Business Costs

In [ ]:
print("="*70)
print("                    BUSINESS COST DEFINITION")
print("="*70)

COST_FN = 200
COST_FP = 20
COST_TP = -80
COST_TN = 0

RETENTION_SUCCESS_RATE = 0.40
AVG_CLV = 200
RETENTION_INCENTIVE_COST = 20

print(f"""
┌──────────────────────────────────────────────────────────────────────────────┐
│                          COST MATRIX                                         │
├──────────────────────────────────────────────────────────────────────────────┤
│                                                                              │
│                              ACTUAL                                          │
│                     Active (0)      Churned (1)                              │
│              ┌────────────────┬────────────────┐                             │
│  PREDICTED   │                │                │                             │
│  Active (0)  │   TN: ${COST_TN:>4}     │   FN: ${COST_FN:>4}    │                             │
│              │   (correct)    │   (COSTLY!)    │                             │
│              ├────────────────┼────────────────┤                             │
│  Churned (1) │   FP: ${COST_FP:>4}     │   TP: ${COST_TP:>4}    │                             │
│              │   (waste)      │   (benefit)    │                             │
│              └────────────────┴────────────────┘                             │
│                                                                              │
│  Key Insight: FN costs 10x more than FP → Favor RECALL                       │
│                                                                              │
└──────────────────────────────────────────────────────────────────────────────┘

Business Assumptions:
  • Average Customer Lifetime Value (CLV): ${AVG_CLV}
  • Retention Incentive Cost: ${RETENTION_INCENTIVE_COST}
  • Retention Success Rate: {RETENTION_SUCCESS_RATE:.0%}
  • Cost Ratio (FN/FP): {COST_FN/COST_FP:.0f}:1
""")

---

## 2. Threshold Optimization

In [ ]:
def calculate_business_cost(y_true, y_proba, threshold, cost_fn, cost_fp, cost_tp, cost_tn):
    """Calculate total business cost for a given threshold"""
    y_pred = (y_proba >= threshold).astype(int)
    
    tn = ((y_true == 0) & (y_pred == 0)).sum()
    fp = ((y_true == 0) & (y_pred == 1)).sum()
    fn = ((y_true == 1) & (y_pred == 0)).sum()
    tp = ((y_true == 1) & (y_pred == 1)).sum()
    
    total_cost = (tn * cost_tn) + (fp * cost_fp) + (fn * cost_fn) + (tp * cost_tp)
    
    return {
        'threshold': threshold,
        'tn': tn, 'fp': fp, 'fn': fn, 'tp': tp,
        'total_cost': total_cost,
        'precision': tp / (tp + fp) if (tp + fp) > 0 else 0,
        'recall': tp / (tp + fn) if (tp + fn) > 0 else 0
    }

thresholds = np.arange(0.05, 0.95, 0.05)
results = []

for thresh in thresholds:
    result = calculate_business_cost(y_test.values, y_proba, thresh, COST_FN, COST_FP, COST_TP, COST_TN)
    results.append(result)

threshold_df = pd.DataFrame(results)

optimal_idx = threshold_df['total_cost'].idxmin()
optimal_threshold = threshold_df.loc[optimal_idx, 'threshold']
optimal_cost = threshold_df.loc[optimal_idx, 'total_cost']

print(f"\n📊 THRESHOLD OPTIMIZATION RESULTS")
print("="*60)
print(f"\nOptimal Threshold: {optimal_threshold:.2f}")
print(f"Expected Cost at Optimal: ${optimal_cost:,.0f}")
print(f"\nAt optimal threshold:")
print(f"   Precision: {threshold_df.loc[optimal_idx, 'precision']:.2%}")
print(f"   Recall: {threshold_df.loc[optimal_idx, 'recall']:.2%}")

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

axes[0, 0].plot(threshold_df['threshold'], threshold_df['total_cost'], 'b-', linewidth=2)
axes[0, 0].axvline(x=optimal_threshold, color='r', linestyle='--', label=f'Optimal: {optimal_threshold:.2f}')
axes[0, 0].axvline(x=0.5, color='gray', linestyle=':', label='Default: 0.50')
axes[0, 0].set_xlabel('Threshold')
axes[0, 0].set_ylabel('Total Business Cost ($)')
axes[0, 0].set_title('Business Cost vs Threshold', fontsize=12, fontweight='bold')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

axes[0, 1].plot(threshold_df['threshold'], threshold_df['precision'], 'g-', label='Precision', linewidth=2)
axes[0, 1].plot(threshold_df['threshold'], threshold_df['recall'], 'b-', label='Recall', linewidth=2)
axes[0, 1].axvline(x=optimal_threshold, color='r', linestyle='--', label=f'Optimal: {optimal_threshold:.2f}')
axes[0, 1].set_xlabel('Threshold')
axes[0, 1].set_ylabel('Score')
axes[0, 1].set_title('Precision & Recall vs Threshold', fontsize=12, fontweight='bold')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

precision, recall, pr_thresholds = precision_recall_curve(y_test, y_proba)
axes[1, 0].plot(recall, precision, 'b-', linewidth=2)
axes[1, 0].set_xlabel('Recall')
axes[1, 0].set_ylabel('Precision')
axes[1, 0].set_title('Precision-Recall Curve', fontsize=12, fontweight='bold')
axes[1, 0].grid(True, alpha=0.3)

opt_precision = threshold_df.loc[optimal_idx, 'precision']
opt_recall = threshold_df.loc[optimal_idx, 'recall']
axes[1, 0].scatter([opt_recall], [opt_precision], color='red', s=100, zorder=5, label=f'Optimal ({opt_recall:.2f}, {opt_precision:.2f})')
axes[1, 0].legend()

default_result = calculate_business_cost(y_test.values, y_proba, 0.5, COST_FN, COST_FP, COST_TP, COST_TN)
optimal_result = calculate_business_cost(y_test.values, y_proba, optimal_threshold, COST_FN, COST_FP, COST_TP, COST_TN)

categories = ['False Negatives\n(Lost Revenue)', 'False Positives\n(Wasted Spend)', 'True Positives\n(Saved Customers)']
default_costs = [default_result['fn'] * COST_FN, default_result['fp'] * COST_FP, default_result['tp'] * abs(COST_TP)]
optimal_costs = [optimal_result['fn'] * COST_FN, optimal_result['fp'] * COST_FP, optimal_result['tp'] * abs(COST_TP)]

x = np.arange(len(categories))
width = 0.35
axes[1, 1].bar(x - width/2, default_costs, width, label='Default (0.5)', color='#e74c3c', alpha=0.7)
axes[1, 1].bar(x + width/2, optimal_costs, width, label=f'Optimal ({optimal_threshold:.2f})', color='#2ecc71', alpha=0.7)
axes[1, 1].set_ylabel('Cost ($)')
axes[1, 1].set_title('Cost Breakdown by Threshold', fontsize=12, fontweight='bold')
axes[1, 1].set_xticks(x)
axes[1, 1].set_xticklabels(categories)
axes[1, 1].legend()

plt.tight_layout()
plt.show()

---

## 3. Cost-Benefit Analysis

In [ ]:
print("="*70)
print("                    COST-BENEFIT ANALYSIS")
print("="*70)

total_churners = y_test.sum()
total_customers = len(y_test)
baseline_cost = total_churners * COST_FN

default_result = calculate_business_cost(y_test.values, y_proba, 0.5, COST_FN, COST_FP, COST_TP, COST_TN)

optimal_result = calculate_business_cost(y_test.values, y_proba, optimal_threshold, COST_FN, COST_FP, COST_TP, COST_TN)

print(f"""
┌──────────────────────────────────────────────────────────────────────────────┐
│                    SCENARIO COMPARISON                                       │
├──────────────────────────────────────────────────────────────────────────────┤
│                                                                              │
│  Test Set: {total_customers:,} customers, {total_churners:,} churners ({y_test.mean():.1%} churn rate)            │
│                                                                              │
├──────────────────────────────────────────────────────────────────────────────┤
│  SCENARIO 1: No Model (Do Nothing)                                           │
│  ─────────────────────────────────                                           │
│  • Churners caught: 0 (0%)                                                   │
│  • Total Cost: ${baseline_cost:,.0f} (all churners lost)                        │
│                                                                              │
├──────────────────────────────────────────────────────────────────────────────┤
│  SCENARIO 2: Model with Default Threshold (0.50)                             │
│  ──────────────────────────────────────────────                              │
│  • Churners caught: {default_result['tp']:,} ({default_result['recall']:.1%})                                  │
│  • False alarms: {default_result['fp']:,}                                                │
│  • Total Cost: ${default_result['total_cost']:,.0f}                                          │
│  • Savings vs No Model: ${baseline_cost - default_result['total_cost']:,.0f}                             │
│                                                                              │
├──────────────────────────────────────────────────────────────────────────────┤
│  SCENARIO 3: Model with Optimal Threshold ({optimal_threshold:.2f})                             │
│  ──────────────────────────────────────────────                              │
│  • Churners caught: {optimal_result['tp']:,} ({optimal_result['recall']:.1%})                                  │
│  • False alarms: {optimal_result['fp']:,}                                                │
│  • Total Cost: ${optimal_result['total_cost']:,.0f}                                          │
│  • Savings vs No Model: ${baseline_cost - optimal_result['total_cost']:,.0f}                             │
│  • Savings vs Default: ${default_result['total_cost'] - optimal_result['total_cost']:,.0f}                              │
│                                                                              │
└──────────────────────────────────────────────────────────────────────────────┘
""")

In [ ]:
print("\n" + "="*70)
print("        WHEN RECALL vs PRECISION DOMINATES")
print("="*70)

print("""
FAVOR RECALL (Lower Threshold) when:
──────────────────────────────────
✓ Cost of missing a churner (FN) is HIGH
✓ Retention incentive cost is LOW
✓ Customer Lifetime Value is HIGH
✓ Retention budget is flexible
✓ Company is in growth mode (every customer matters)

Example: FN cost = $200, FP cost = $20 → Favor recall

FAVOR PRECISION (Higher Threshold) when:
────────────────────────────────────────
✓ Retention budget is LIMITED
✓ Incentives are EXPENSIVE (>$100 per customer)
✓ False positives annoy customers (bad CX)
✓ Company needs to optimize ROI strictly
✓ High-touch retention (expensive human intervention)

Example: FN cost = $100, FP cost = $100 → Balance needed

OUR SITUATION:
──────────────
• Cost ratio (FN/FP) = {COST_FN}/{COST_FP} = {COST_FN/COST_FP:.0f}:1
• This strongly favors RECALL
• Optimal threshold ({optimal_threshold:.2f}) is BELOW 0.5 → Recall-biased
""")

---

## 4. ROI Calculation

In [ ]:
print("="*70)
print("                    RETURN ON INVESTMENT (ROI)")
print("="*70)

churners_caught = optimal_result['tp']
false_alarms = optimal_result['fp']
churners_saved = churners_caught * RETENTION_SUCCESS_RATE

revenue_saved = churners_saved * AVG_CLV
retention_cost = (churners_caught + false_alarms) * RETENTION_INCENTIVE_COST
net_benefit = revenue_saved - retention_cost
roi = (net_benefit / retention_cost) * 100 if retention_cost > 0 else 0

print(f"""
┌──────────────────────────────────────────────────────────────────────────────┐
│                        ROI CALCULATION                                       │
├──────────────────────────────────────────────────────────────────────────────┤
│                                                                              │
│  INPUTS (at threshold = {optimal_threshold:.2f})                                              │
│  ─────────────────────────────────                                           │
│  • Churners identified (TP):        {churners_caught:>6,}                                    │
│  • False alarms (FP):               {false_alarms:>6,}                                    │
│  • Total targeted:                  {churners_caught + false_alarms:>6,}                                    │
│                                                                              │
│  RETENTION OUTCOMES                                                          │
│  ─────────────────                                                           │
│  • Retention success rate:          {RETENTION_SUCCESS_RATE:>6.0%}                                    │
│  • Customers saved:                 {churners_saved:>6,.0f}                                    │
│                                                                              │
│  FINANCIAL IMPACT                                                            │
│  ────────────────                                                            │
│  • Revenue saved:                   ${revenue_saved:>10,.0f}                              │
│  • Retention cost:                  ${retention_cost:>10,.0f}                              │
│  • NET BENEFIT:                     ${net_benefit:>10,.0f}                              │
│                                                                              │
│  ══════════════════════════════════════════════════════════════════════════  │
│  │          RETURN ON INVESTMENT (ROI): {roi:>6.0f}%                       │  │
│  ══════════════════════════════════════════════════════════════════════════  │
│                                                                              │
└──────────────────────────────────────────────────────────────────────────────┘
""")

scale_factor = 5
annual_benefit = net_benefit * scale_factor * 4
print(f"\n📈 Estimated Annual Impact (scaled): ${annual_benefit:,.0f}")

In [ ]:
cost_analysis = {
    'optimal_threshold': optimal_threshold,
    'precision_at_optimal': optimal_result['precision'],
    'recall_at_optimal': optimal_result['recall'],
    'total_cost_at_optimal': optimal_result['total_cost'],
    'net_benefit': net_benefit,
    'roi': roi,
    'cost_fn': COST_FN,
    'cost_fp': COST_FP,
    'retention_success_rate': RETENTION_SUCCESS_RATE
}

pd.DataFrame([cost_analysis]).to_csv(
    os.path.join(PROCESSED_DATA_PATH, 'cost_analysis.csv'), index=False
)
print(f"\n✅ Cost analysis saved to {PROCESSED_DATA_PATH}/cost_analysis.csv")

---

##  Phase 6 Checklist

- [x] Defined business cost matrix (FN, FP, TP, TN)
- [x] Calculated expected cost at different thresholds
- [x] Found optimal decision threshold
- [x] Compared cost vs default threshold
- [x] Explained when recall vs precision dominates
- [x] Calculated ROI of the model
- [x] Saved cost analysis results

**Phase 6 Status: COMPLETE** 

---

**Next**: Phase 7 - User Segmentation & Retention Strategy